##Imports

In [0]:
#data
import numpy as np
import pandas as pd
import statistics
import math

#images
import skimage
from PIL import Image, ImageEnhance
from keras.preprocessing import image

#visualization
import matplotlib.pyplot as plt

#model
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split

In [0]:
#access drive files
from google.colab import drive
drive.mount("/content/drive", force_remount = True)

##Images

In [0]:
#determine the correct letter for the filename
def determine_letter(l):
    if l == 0:
        return "A"
    if l == 1:
        return "B"
    if l == 2:
        return "C"
    if l == 3:
        return "D"
    if l == 4:
        return "E"
    if l == 5:
        return "F"
    if l == 6:
        return "G"
    else:
        return "H"

In [0]:
#determine whether the filename includes "Phase" or "Default0"
def determine_word(p, d, l, n):
    if l == "A" or l == "E":
        if passage == "7" and diet == "STD":
          return "Default0"
        else:
          return "Phase"
        
    if passage == "1" and diet == "STD":
      return "Default0"
    
    if n == 5 or n == 6 or n == 7 or n == 8:
      return "Phase"
    
    if passage == "7":
        if diet == "STD" or n == 1 or n == 2 or n == 3 or n == 4:
          return "Default0"
        else:
          return "Phase"
    
    if n == 9 or n == 10 or n == 11 or n == 12:
      return "Default0"
    
    else:
      return "Phase"

In [0]:
#put all the images in an array
images_array = []

for p in range(0,2): #passage
    for d in range(0,2): #diet
        for l in range(0,2): #letter
            for n in range(1,9): #number
                image_set = [] #each set has 25 images
            
                for i in range(0,25): #image number
                    if p == 0:
                        passage = "1"
                    else:
                        passage = "7"
                    if d == 0:
                        diet = "STD"
                    else:
                        diet = "MD"

                    letter = determine_letter(l)

                    #open image
                    im = Image.open(#insert path to folder with images
                                    + "/Passage" + passage + "/" + diet + "/" + letter + str(n) + 
                                    "/img_000000001_" + determine_word(passage, diet, letter, n) + 
                                    "_0" + "{0:0=2d}".format(i*2) + ".tif")

                    #resize image
                    im = im.resize((640, 540))

                    #convert to array
                    im = image.img_to_array(im)

                    #convert to three channels
                    im = np.concatenate((im, im, im))

                    #append to image set array
                    image_set.append(im)
                    
                images_array.append(image_set)

In [0]:
#convert to dataframe
image_cols = ["0", "2", "4", "6", "8", "10", "12", "14", "16", 
              "18", "20", "22", "24", "26", "28", "30", "32",
              "34", "36", "38", "40", "42", "44", "46", "48"]

##Diversity Indexes

In [0]:
#access the diversity spreadsheet
diversity_sheet = pd.read_excel(#insert path to folder with spreadsheet + 
                                "/Diversity.xlsx")

#drop all columns except for diets, wells, and shannon diversity indexes
diversity_sheet = diversity_sheet.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4",
                                          "Unnamed: 5", "T7", "Unnamed: 8", 
                                           "T1", "Unnamed: 11"], axis = 1)

#rename the remaining columns
diversity_sheet = diversity_sheet[1:]
diversity_sheet.columns = ["diet", "well", "p7 diversity", "p1 diversity"]

In [0]:
#data frame without diversities
sheet = diversity_sheet.drop(["p1 diversity", "p7 diversity"], axis = 1)

#data frame for p1 diversities
diversity_sheet_p1 = sheet.copy()
diversity_sheet_p1["passage"] = "1"
diversity_sheet_p1["diversity"] = diversity_sheet["p1 diversity"]

#data frame for p7 diversities
diversity_sheet_p7 = sheet.copy()
diversity_sheet_p7["passage"] = "7"
diversity_sheet_p7["diversity"] = diversity_sheet["p7 diversity"]

#combine the data frames
diversity_sheet = diversity_sheet_p1.append(diversity_sheet_p7, 
                    ignore_index = True)

In [0]:
#uses A1-A8, B1-B8
diversity_sheet = diversity_sheet[diversity_sheet["well"].str.contains("A")].append(diversity_sheet[diversity_sheet["well"].str.contains("B")])
diversity_sheet = diversity_sheet[~diversity_sheet["well"].str.contains("11")]
diversity_sheet = diversity_sheet[~diversity_sheet["well"].str.contains("12")]
diversity_sheet = diversity_sheet[~diversity_sheet["well"].str.contains("10")]
diversity_sheet = diversity_sheet[~diversity_sheet["well"].str.contains("9")]

In [0]:
#add the image columns to the diversity dataframe
data = diversity_sheet.copy()
data[image_cols] = pd.DataFrame(images_array, index = data.index)

In [0]:
#drop all rows with shannon diversity index of 0
data = data[data["diversity"] != 0]

##Data Prep

In [0]:
#split into input/output arrays
X = np.array(data[image_cols])
Y = np.array(data["diversity"])

In [0]:
#split the data with 60% for training, 40% for validation/testing
train_X, validtest_X, train_Y, validtest_Y = train_test_split(X, Y, test_size = .20, 
                                                       random_state = 128)

#split the testing/validation data in half; 20% overall for each
valid_X, test_X, valid_Y, test_Y = train_test_split(validtest_X, validtest_Y, test_size = .50, 
                                                       random_state = 128)

In [0]:
#preps the x data by combining all 25 image columns into one column &
#putting image pixels into an array/normalizes image data

def prep_x(x):
    new_x = x[:,0]
    
    for index in range(len(x)):
        new_index = index*25
        to_insert = x[index, 1:]
        to_insert = to_insert.T #turn the row into a column
        new_x = np.insert(new_x, new_index + 1, to_insert)
    
    x = new_x
    
    #turn x into an array of pixel arrays
    x = np.array(x.tolist())
    
    #reshape array
    x = x.reshape(-1, 540, 640, 3)
    
    #convert pixel values to float32 and 0 to 1 valuues
    x = (x/4095).astype('float32')
    
    #mean 0, stddev 1
    x = (x - x.mean())/x.std()
    
    return x

In [0]:
#single column & prep images
train_X = prep_x(train_X)
valid_X = prep_x(valid_X)
test_X = prep_x(test_X)

In [0]:
#preps the y data by copying the diversity indexes so that 
#each image has a corresponding output

def prep_y(y):
    length = len(y) #to make sure max index for loop doesn't change
    
    for index in range(length):
        new_index = index*25
        rows = [y[new_index]] * 24
        y = np.insert(y, new_index+1, rows)
    
    return y

In [0]:
#prep y data
train_Y = prep_y(train_Y)
valid_Y = prep_y(valid_Y)
test_Y = prep_y(test_Y)

##Training The Model

In [0]:
#define the base model
base_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (540, 640, 3), channel=1)

#freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False
    
#create model
model = Sequential()

#add the base model
model.add(base_model)

#add layers for regression
model.add(Flatten())
model.add(Dense(1, activation= 'linear'))

#model summary
#model.summary()

In [0]:
#concordance index metric
def c_index_metric(y_true, y_pred):
  
  #compares predicted values with each other
  s1 = tf.less(tf.expand_dims(y_pred, -1), y_pred)
  s2 = tf.equal(tf.expand_dims(y_pred, -1), y_pred)
  s = tf.cast(s2, tf.float32) * 0.5 + tf.cast(s1, tf.float32)

  #compares true values with each other
  n = tf.less(tf.expand_dims(y_true, -1), y_true)
  n = tf.cast(n, tf.float32)

  #concordant pairings
  s = tf.reduce_sum(tf.multiply(s,n))

  #total pairings
  n = tf.reduce_sum(n)

  #concordant pairings/total pairings
  return tf.where(tf.equal(s,0), 0.0, s/n)

In [0]:
#define the optimizer
sgd = keras.optimizers.SGD(lr = 0.00001, momentum = 0.9, decay = 0.0, nesterov = True)

#compile the model
model.compile(loss = keras.losses.mean_squared_error, optimizer = sgd, metrics = [c_index_metric])

In [0]:
#train the model
hist = model.fit(train_X, train_Y, epochs = 50, batch_size = 32, validation_data = (valid_X, valid_Y))

In [0]:
#plot validation concordance over training epochs
plt.plot(hist.history['val_c_index_metric'])
title = plt.title('concordance vs epoch')
plt.setp(title, color = 'w')
plt.tick_params(colors = 'w')
plt.show()

##Evaluating The Model

In [0]:
#get array of true values
y_true = np.array(test_Y)

#get array of predicted values
y_pred = model.predict(test_X)
y_pred = y_pred[:,0]

#for index in range(len(y_true)):
   #print ("true: "+ str(y_true[index]) + " predicted: " + str(y_pred[index]) + "\n")

In [0]:
#determine the predictions' loss & concordance
evaluation = model.evaluate(test_X, test_Y, verbose = 0)
loss = evaluation[0]
print("loss: " + str(loss) + "\n" + "concordance: " + str(evaluation[1])) 

In [0]:
#determine 95% confidence interval
#print("95% CI: " + str(loss) + " +/- " + str(1.96* math.sqrt((loss * (1-loss))/325)))

In [0]:
#predicted vs true values graph
plt.scatter(y_true, y_pred)

#title
title = plt.title("predictions vs true values")
plt.setp(title, color = 'w')

#ticks
plt.tick_params(colors = 'w')

#display graph
plt.show()

In [0]:
#determine concordance index for testing data
def c_index(gt, pred, **kwargs):
    '''Basic implementation for concordance index.
    We want score improvement, hence only compare y1 < y2.
    This implementation should handle continuous prediction/ground truth
    Arguments:
        gt (list(float)): ground truth
        pred (list(float)): predictions
    '''

    assert len(gt) == len(pred), 'Ground truth must have same size as predictions'
    s = 0
    n = 0

    for i, y1 in enumerate(gt):
        for j, y2 in enumerate(gt):
            if y1 < y2:
                s += (pred[i] < pred[j]) + 0.5 * (pred[i] == pred[j])
                n += 1

    if n == 0:
        if 'throw_error' in kwargs:
            raise ValueError(f'All ground truth values equal to {gt[0]}')
        else:
            return 0

    return s / n

In [0]:
print("concordance index for testing data: " + str(c_index(y_true.tolist(), y_pred.tolist())))

In [0]:
#to locate a specific value in a list
def locate_val(vals, val):
  for x in range(len(vals)):
    if vals[x] == val:
      return x

In [0]:
#taking the mean of all the outputs for each sample

true_vals = []
pred_vals = []
count = []

for i in range(len(y_true)):
  
  #if the true value not already in array
  if not (y_true[i] in true_vals):
    
    true_vals.append(y_true[i])
    pred_vals.append(y_pred[i])
    count.append(1)
  
  #if true value already in array
  else:
    #index of true value in the true_vals array
    index = locate_val(true_vals, y_true[i])
    
    #finding the mean of the predictions with the same true value; updating pred_vals
    pred_vals[index] = ((pred_vals[index]*count[index]) + y_pred[index])/(count[index] + 1)
    
    #updating the count of the predicted values already found for the same true value
    count[index]+=1

In [0]:
print("concordance index for testing data (mean): " + str(c_index(true_vals, pred_vals)))